In [1]:
run FunctionsLoader.ipynb

0.2.0_4
0.2.0_4


In [2]:
class Predictions():
    
    
    def __init__(self, min_days=30):
        self.min_days = min_days
            
        
    # time_window should be in mins (e.g; 30mins)
    # sd_ratio should be between 0 and 1 (e.g; 0.25 for using 1/4 of sd)

    def loadData(self, time_window, sd_ratio, t_hist):
        
        ### read mobility data ###
        ld = LocationData()
        ld.readLocationData(precision_value=4) # read file
        ld.filterUser(self.min_days) #filter users with min 30 days
        #print len(ld.users)
        ld.convertToEqualTimeSeries(time_window * 60 * 1000) # convert to time series of 30mins
        dt = ld.createUserDailyTrajectoryLists() # create daily trajectories of users 

        ### read mood data ###
        pd = PhqData()
        pd.readPhqData()
        pd.computeScores(ld.users, self.min_days)
        pd.computeDivergence()
        pd.convertTo2Labels(sd_ratio=sd_ratio) 
        #len(list(set(m[0] for m in pd.phq_scores)))

        ### merge data ###
        mlm = PhqLocMerge()
        mlm.mergeData(dt, pd.phq_scores, t_hist)
        
        ### set data as a class variable ###
        self.data = mlm.data

    
    
    def prepareInput(self):
    
        ###  transform mobility data ### 
        users = list(set([m[0] for m in self.data]))
        dt2 = approachAvg2(lm_data=self.data, users=users) # Computing displacement changes
        dt3 = approachAvg3(lm_data=self.data, users=users) # Computing displacement [normalised]
        dt4 = approachAvg4(lm_data=self.data, users=users, topn=20) # Computing displacement [normalised]

        ###  combinations of data for using different combinations of features ### 
        self.data1 = [dt2] 
        self.data2 = [dt3] 
        self.data3 = [dt4] 
        self.data4 = [dt2,dt3] 
        self.data5 = [dt2,dt4] 
        self.data6 = [dt3,dt4] 
        self.data7 = [dt2,dt3,dt4]
    
    
    def computePredictions(self, is_generic=False, predict_raw_score=False, layers=5, act_func=''):

        ### filter users ### 
        filtered_users = []
        users = list(set([m[0] for m in self.data]))
        for u in users:
            ud = [d for d in self.data1[0] if d[0]==u]
            if len(ud) > self.min_days:
                filtered_users.append(u)
        print 'Number of users', len(filtered_users)
        
        ### computing prdiction accuracy ### 
        if is_generic:
            model1 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data1, iteration=20, users=filtered_users, min_days=self.min_days) 
            model2 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data2, iteration=20, users=filtered_users, min_days=self.min_days) 
            model3 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data3, iteration=20, users=filtered_users, min_days=self.min_days) 
            model4 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data4, iteration=20, users=filtered_users, min_days=self.min_days) 
            model5 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data5, iteration=20, users=filtered_users, min_days=self.min_days) 
            model6 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data6, iteration=20, users=filtered_users, min_days=self.min_days) 
            model7 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data7, iteration=20, users=filtered_users, min_days=self.min_days) 

        else:
            model1 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data1, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model2 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data2, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model3 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data3, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model4 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data4, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model5 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data5, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model6 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data6, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model7 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data7, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 

        self.models = [model1, model2, model3, model4, model5, model6, model7]

        
    def plotPredictions(self, include_mt=False, t_hist=14, file_path=''):
        model_names = ['IR 1','IR 2','IR 3','IR 1+2','IR 1+3','IR 2+3','IR 1+2+3']
        label_dict = {0:'R2', 1:'MSE'}
        Plots.multiModelRawScores(models=p.models, model_names=model_names, label_dict=label_dict, include_mt=include_mt, t_hist=t_hist, file_path=file_path)
        
        


In [ ]:
all_models = [] # store models of all iterations (for debugging)
for layers in [5,7,9]:
    for time_window in [10,30,60]:
        for t_hist in [1,7,14]:
            print 'Iteration values:', time_window, t_hist
            p = Predictions(30)
            p.loadData(time_window=time_window, sd_ratio=0.25, t_hist=t_hist)
            p.prepareInput()
            p.computePredictions(is_generic=False, predict_raw_score=True, layers=layers, act_func='tanh')
            all_models = p.models
            file_path = 'plots/regression/tanh/raw_score_prediction_time_window_' + str(time_window) + '_t_hist_' + str(t_hist)  + '_ae_' + str(layers)  + '.pdf'
            p.plotPredictions(include_mt=True, t_hist=t_hist, file_path=file_path)

Iteration values: 10 1
Initial user count 6705
User count 5616
Min time ratio 0.500023134225
Number of users 44
Pos count: 900
Neg count: 1489
Number of users 23
Ecoded X data
Performance computed:  -0.307285273627 12.4479448407
Performance computed:  -0.167025191443 8.5972163544
Performance computed:  -0.187241062781 10.6993075243
Performance computed:  -0.142863207564 9.7486111693
Performance computed:  -0.237667178499 13.1729148491
Performance computed:  -0.148762215207 14.7056735419
Performance computed:  -0.163577297126 13.668312522
Performance computed:  -0.426024845774 10.3094449313
Performance computed:  -0.163965162266 12.6891149298
Performance computed:  -0.333530600947 10.8778877433
Performance computed:  -0.0927760700115 9.19719930368
Performance computed:  -0.437668921339 11.6911402078
Performance computed:  -0.0851102145334 9.9159886417
Performance computed:  -0.164101356361 11.1425337956
Performance computed:  -0.0347321274186 9.36591009689
Performance computed:  -0.1921

Performance computed:  -0.539598484055 14.8249511128
Performance computed:  0.0723180740604 10.8641675357
Performance computed:  -0.0744102454903 6.45890884566
Performance computed:  -0.118774202412 11.3205886324
Performance computed:  0.0118462849956 11.6991650421
Performance computed:  -0.207740727519 15.6422879677
Performance computed:  -0.638833995354 16.808698442
Performance computed:  -0.170216322424 9.93584575939
Performance computed:  -0.475948945899 11.1301816113
Performance computed:  -0.989480147672 16.1234011056
Performance computed:  0.04904825177 9.28327249493
Performance computed:  0.0995941947907 7.53379352954
Performance computed:  -0.482733892724 12.0480426273
Performance computed:  -0.676164707398 14.7847147546
Performance computed:  -0.351360744881 13.5763913042
Performance computed:  -0.313797389053 15.1752915984
Performance computed:  -0.115046194527 11.3467338936
Performance computed:  0.0255200131928 10.0704034265
Performance computed:  -0.171307892531 12.026635

Performance computed:  -0.483797286953 8.80824497727
Performance computed:  -0.406013118703 8.74623513174
Performance computed:  -0.28478162942 6.01778702853
Performance computed:  -0.747894477222 9.48560962891
Performance computed:  -0.463144994613 9.47404628426
Performance computed:  -0.376285329253 5.98079896452
Performance computed:  -0.0214435256949 4.66132279198
Performance computed:  -0.136173020265 7.14805751464
Performance computed:  -0.45900589862 9.14802222491
Performance computed:  -0.204193882101 8.13458394364
Performance computed:  -0.367153887082 8.4133865698
Performance computed:  -0.469225855363 9.34734744732
Performance computed:  -0.336508148607 5.48269529261
Performance computed:  -0.178379470113 2.27373777994
Performance computed:  -0.730557259693 2.00775580576
Performance computed:  -0.325235988547 1.68303951525
Performance computed:  -0.688728063014 2.88507275625
Performance computed:  0.0417097710286 1.25893030081
Performance computed:  -0.431984494998 2.2005544

Performance computed:  -1.327183739 2.00818954646
Performance computed:  -0.962103332433 4.116041131
Performance computed:  -0.709106542588 1.89075995751
Performance computed:  -1.06834590341 1.54829557324
Performance computed:  -0.194335946143 2.23739526339
Performance computed:  -0.195954718431 5.7240190816
Performance computed:  -0.646933562002 6.52290351355
Performance computed:  -0.34944192411 4.00094815581
Performance computed:  -0.636574897717 5.79810203821
Performance computed:  -0.206609505704 5.60272416621
Performance computed:  -0.0631454481035 4.33336458515
Performance computed:  -0.356608013388 7.53427481782
Performance computed:  -0.725815602713 7.13619499406
Performance computed:  -0.387967840172 6.61691683929
Performance computed:  -0.497643546371 6.66480655417
Performance computed:  -0.15793546913 6.34690481408
Performance computed:  -0.155770486311 5.18753143025
Performance computed:  0.127916016499 6.08580237966
Performance computed:  0.187082278457 4.2376996729
Perf

Performance computed:  -8.02614544409 1.05696789966
Performance computed:  -0.197637514224 4.0978692602
Performance computed:  0.580475511444 0.69172845805
Performance computed:  0.00701509737895 0.985461522109
Performance computed:  -0.700216106144 6.83773998488
Performance computed:  -0.320345851368 11.2084275865
Performance computed:  -1.119181985 4.61720080625
Performance computed:  -0.200959126376 5.96762271238
Performance computed:  -0.431929204671 7.04530314483
Performance computed:  -0.245945853476 6.42267550791
Performance computed:  -1.00668158297 10.8333369981
Performance computed:  -1.42665009961 8.82660757232
Performance computed:  -0.0516719480446 6.67701504844
Performance computed:  -0.994684847082 6.49533040152
Performance computed:  -0.238575978884 6.07750704322
Performance computed:  -0.288648238276 6.38668544859
Performance computed:  -0.705073628405 8.39132298037
Performance computed:  0.0323092910476 4.89913860831
Performance computed:  -0.507954105745 4.5896786458

...

Performance computed:  -1.24982060605 20.4896544465
Performance computed:  -0.168637338086 11.3508614195
Performance computed:  -0.739602107027 18.0205791459
Performance computed:  -0.594025591152 18.054110449
Performance computed:  -1.22186560572 15.7923838976
Performance computed:  0.0489551130037 13.8428370038
Performance computed:  0.0251510801068 13.419135824
Performance computed:  -0.536607362729 14.9742591313
Performance computed:  -0.494659689746 23.3231543707
Performance computed:  -1.72637814986 16.9631450111
Performance computed:  0.092088652782 12.4709540393
Performance computed:  0.207802858349 8.72378458751
Performance computed:  -0.107679743021 15.9187934164
Performance computed:  -1.02077162357 21.9528163291
Performance computed:  -0.123817121686 13.2278639954
Performance computed:  -0.164743189728 17.2503731492
Performance computed:  -0.206459116989 15.6936844497
Performance computed:  -0.196936503209 18.8898963464
Performance computed:  -0.207407010085 11.8155665344
P

Performance computed:  -0.169953232462 0.57860952381
Performance computed:  0.356132386732 0.495617460317
Performance computed:  -0.435725758121 0.543231746032
Performance computed:  -1.36000347102 0.925057142857
Performance computed:  -0.214221326845 9.66849027778
Performance computed:  -0.923446085326 11.3964180556
Performance computed:  0.0428953886451 3.61597777778
Performance computed:  -3.59106182432 9.72468194444
Performance computed:  -0.25438639345 6.25866944444
Performance computed:  -0.64996404019 8.66689444444
Performance computed:  -0.646104832563 5.21172222222
Performance computed:  -0.309181605408 6.95311805556
Performance computed:  -6.28551325961 5.17018472222
Performance computed:  -0.348026886371 8.35980277778
Performance computed:  -1.35178674958 7.56356666667
Performance computed:  -1.56344988093 9.86714583333
Performance computed:  -2.42736344979 9.62589305556
Performance computed:  -1.15186205053 8.94337777778
Performance computed:  -0.0257107313082 5.85451111111

Performance computed:  0.260064311353 0.816390564374
Performance computed:  0.0384844034223 1.12372371378
Performance computed:  0.0112216001772 1.30467152431
Performance computed:  -0.757023512723 1.36687929705
Performance computed:  0.115757288622 1.03941815067
Performance computed:  0.0483692504607 0.80894521542
Performance computed:  -0.0446079651321 1.23987053918
Performance computed:  -0.0544849382216 1.1060492517
Performance computed:  -0.992322863745 1.35929854119
Performance computed:  -0.261985025842 1.2426062585
Performance computed:  -0.240113811123 1.61316029227
Performance computed:  -1.00872707083 1.92842702948
Performance computed:  -0.167793678962 1.1806491484
Performance computed:  -0.0453580506835 1.16530755354
Performance computed:  -0.288519941792 1.12164310406
Performance computed:  -0.462520685745 1.43441143613
Performance computed:  0.36332771182 0.782522071051
Performance computed:  0.286726315232 0.649874167297
Performance computed:  0.184333243707 1.113766835

Performance computed:  -1.43075853587 2.97110358676
Performance computed:  -0.887237383182 1.89520149177
Performance computed:  -0.38473649727 4.11501239069
Performance computed:  -0.124442094723 1.41236564429
Performance computed:  -0.213179641317 2.39941117622
Performance computed:  -0.425212863168 3.06272206629
Performance computed:  -0.620860912089 3.40909822531
Performance computed:  -0.369047886827 2.59797868441
Performance computed:  -0.0548091231751 3.38073016011
Performance computed:  -0.286525709816 2.09614348637
Performance computed:  -0.746982412823 3.76480401436
Performance computed:  -0.249507753645 3.53382651618
Performance computed:  0.257191147026 1.38994365184
Performance computed:  -0.314451611009 2.9366126929
Performance computed:  1.0 0.0
Performance computed:  1.0 0.0
Performance computed:  1.0 0.0
Performance computed:  1.0 0.0
Performance computed:  1.0 0.0
Performance computed:  1.0 0.0
Performance computed:  1.0 0.0
Performance computed:  1.0 0.0
Performance c

Performance computed:  -0.0959114509668 6.2552
Performance computed:  -0.427990280024 8.84538333333
Performance computed:  -0.0288296758647 4.52140277778
Performance computed:  -0.0407049109195 4.20558611111
Performance computed:  -0.143328909088 4.44165416667
Performance computed:  0.170294379564 0.566144444444
Performance computed:  -0.595309830345 10.3525138991
Performance computed:  -0.890651720729 8.5121741187
Performance computed:  -0.0548119755576 8.12656327378
Performance computed:  -0.0432641968792 4.0347304074
Performance computed:  0.151556853847 5.31923620269
Performance computed:  -0.130875139095 7.16964146664
Performance computed:  -0.108621592089 3.23255841648
Performance computed:  -0.204837500421 6.06257264504
Performance computed:  -0.0404270108491 7.17197738261
Performance computed:  -0.182969670205 5.52321911958
Performance computed:  0.0684517099575 6.29540475329
Performance computed:  -0.187381941389 9.31367117512
Performance computed:  0.220872695884 4.6025497521

In [4]:
# mn = ['IR 1','IR 2','IR 3','IR 1+2','IR 1+3','IR 2+3','IR 1+2+3']
# ld = {1:'Sensitivity', 0:'Specificity'}
# Plots.multiModel(models=p.models, model_names=mn, label_dict=ld, include_mt=True, t_hist=7, file_path='plots/prediction_plot.png')